In [1]:
import re
import base64
from github import Github

def scan_for_api_keys(content):
    # Example pattern for API keys, adjust this regex pattern as needed
    api_key_pattern = re.compile(r'[A-Za-z0-9]{32,}')
    return re.findall(api_key_pattern, content)

def main():
    github_token = ''
    g = Github(github_token)
    user = g.get_user()

    for repo in user.get_repos():
        print(f'Scanning repository: {repo.name}')

        try:
            for file in repo.get_contents(''):
                if file.type == 'file':
                    content = base64.b64decode(file.content).decode('utf-8')
                    keys = scan_for_api_keys(content)

                    if keys:
                        print(f'  - Found potential API keys in {file.name}: {keys}')
                else:
                    print(f'  - Skipping {file.name} (not a file)')

        except Exception as e:
            print(f'  - Error scanning {repo.name}: {e}')

if __name__ == '__main__':
    main()

BadCredentialsException: 401 {"message": "Bad credentials", "documentation_url": "https://docs.github.com/rest"}

In [3]:
import re
import base64
import requests

def scan_for_api_keys(content):
    # Example pattern for API keys, adjust this regex pattern as needed
    api_key_pattern = re.compile(r'[A-Za-z0-9]{32,}')
    return re.findall(api_key_pattern, content)

def main():
    github_username = 'Joshua-Stapleton'
    repos_url = f'https://api.github.com/users/{github_username}/repos'

    response = requests.get(repos_url)

    if response.status_code == 200:
        repositories = response.json()

        for repo in repositories:
            print(f'Scanning repository: {repo["name"]}')

            try:
                contents_url = repo["contents_url"].replace('{+path}', '')
                contents_response = requests.get(contents_url)

                if contents_response.status_code == 200:
                    files = contents_response.json()

                    for file in files:
                        if file["type"] == "file":
                            file_response = requests.get(file["download_url"])

                            if file_response.status_code == 200:
                                content = file_response.text
                                keys = scan_for_api_keys(content)

                                if keys:
                                    print(f'  - Found potential API keys in {file["name"]}: {keys}')
                        else:
                            print(f'  - Skipping {file["name"]} (not a file)')

            except Exception as e:
                print(f'  - Error scanning {repo["name"]}: {e}')
    else:
        print('Failed to fetch repositories. Check your username and try again.')


main()

Scanning repository: bugfinder_tools
Scanning repository: diem_cvs
  - Skipping tools (not a file)
Scanning repository: tax_optimisation
  - Skipping functions (not a file)
